In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import nct, ncx2
from scipy.special import hyp1f1
import sys
from scipy.optimize import brentq

In [ ]:
# def lambda_max_generator(eps, y, theta, sigma, kappa):

#   # Define lambda function for root finding
#   G1 = lambda H : hyp1f1(H/kappa, (2 * kappa * theta)/(sigma**2), (2 * kappa * y)/(sigma**2))/hyp1f1(H/kappa, \
#     (2 * kappa * theta)/(sigma**2), (2*kappa*H)/(sigma**2)) - eps
#   F1 = lambda H : (hyp1f1(H/kappa, (2 * kappa * theta)/(sigma**2), (2 * kappa * y)/(sigma**2))/hyp1f1(H/kappa, \
#     (2 * kappa * theta)/(sigma**2), (2*kappa*H)/(sigma**2)) - eps)**2
#   # Root finding
#   Hstar = scipy.optimize.toms748(G1,y,)#scipy.optimize.minimize(F1, x0 = y,method='L-BFGS-B', bounds=[(y, None)]).x[0]
#   while G1(Hstar) > 0:
#     Hstar = Hstar + 1e-8
#   return Hstar


In [ ]:
from scipy.optimize import brentq

def lambda_max_generator(epsilon, y, theta, sigma, kappa, max_attempts=5, tol=1e-8):
    """
    Compute H^*_epsilon such that P_y(sigma_H < tau) = G_y(H; H) <= epsilon.
    This function implements Equation (11) using the closed-form in Equation (19).
    """

    def G_y(H):
        a = H / kappa  # Now using H as the Laplace exponent (since tau ~ Exp(H))
        b = 2 * kappa * theta / sigma**2
        z_y = 2 * kappa * y / sigma**2
        z_H = 2 * kappa * H / sigma**2
        num = hyp1f1(a, b, z_y)
        denom = hyp1f1(a, b, z_H)
        return num / denom  # This is G_y(H; H) = E[e^{-H * sigma_H}]

    def root_function(H):
        return G_y(H) - epsilon

    # Bracketing interval
    H_min = y + 1e-8
    H_max = y + 10.0

    # Expand H_max until G_y(H_max) < epsilon
    for _ in range(max_attempts):
        try:
          H_star = scipy.optimize.toms748(root_function, H_min, H_max, xtol=tol)
          if root_function(H_star)<=0:
            return H_star
          else:
            while root_function(H_star)>0:
              H_star = H_star + 1e-2
            return H_star
        except Exception:
            pass
        H_max += 5.0

    # Fallback: scan manually to find conservative H
    print("[Warning] brentq failed to converge. Using fallback grid search.")
    H_vals = np.linspace(H_min, H_max + 50, 1000)
    for H in H_vals:
        if G_y(H) <= epsilon:
            return H

    raise RuntimeError("Unable to find H^*_epsilon. Try expanding search space.")

In [ ]:

import numpy as np
from scipy.stats import ncx2

def simulate_modified_cir_jump(kappa, theta, sigma, T, delta, omega, eps, lambda_benchmark, low=0.0, high=1.0):
    t = 0.0
    lambda_t = theta.copy()
    gamma = theta.copy()
    events = []
    marks = []
    survival_prob = 0
    n = len(theta)
    Dn = np.arange(1, n)  # skip index 0 (assumed systemic)
    indices = np.arange(n)

    #Precompute lambda_benchmark once
    # lambda_benchmark = np.array([
    #     lambda_max_generator(eps, lambda_t[i], theta[i], sigma[i], kappa[i]) for i in range(n)
    # ])


    while t < T:
        # Compute lambda_max
        mask = lambda_t <= gamma
        lambda_max = np.where(mask, lambda_benchmark, lambda_t + lambda_benchmark - gamma)

        systemic_part = lambda_max[0] * np.sum(omega[Dn - 1])
        idiosyncratic_part = np.sum(lambda_max[Dn])
        Hepsilon = systemic_part + idiosyncratic_part
        print(Hepsilon)

        tau = np.random.exponential(1 / Hepsilon)
        t_proposed = t + tau
        if t_proposed >= T:
            break

        exp_kappa_tau = np.exp(-kappa * tau)
        one_minus_exp = 1 - exp_kappa_tau
        c = sigma**2 * one_minus_exp / (4 * kappa)
        df = 4 * kappa * theta / (sigma**2)
        nc = 4 * kappa * lambda_t * exp_kappa_tau / (sigma**2 * one_minus_exp)

        lambda_proposedf = c * ncx2.rvs(df, nc, size = len(df))
        lambda_proposed = lambda_proposedf[0] * np.sum(omega[Dn - 1]) + np.sum(lambda_proposedf[Dn])

        if np.random.uniform() < min(lambda_proposed / Hepsilon, 1):
            # Accept
            t = t_proposed
            events.append(t)

            probs = (omega[Dn - 1] * lambda_proposedf[0] + lambda_proposedf[Dn]) / lambda_proposed
            selected_idx = np.random.choice(len(Dn), p=probs)
            selected_defaulter = Dn[selected_idx]
            Dn = np.delete(Dn, selected_idx)

            mark = np.random.uniform(low, high)
            marks.append(mark)
            indices = np.concatenate(([0], Dn))
            lambda_t[indices] = lambda_proposedf[indices] + delta[indices] * mark
        else:
            # Reject
            t = t_proposed
            lambda_t = lambda_proposedf

    if 1 in Dn:
      survival_prob = 1
    return np.array(events), np.array(marks), survival_prob


In [ ]:
num_trials = 2500
T = 5
TrueValue = 5.4494
Payoff_T = np.zeros(num_trials)
eps =  1/num_trials
survival_probability = 0
mean_survival_probability = 0



for i in range(num_trials):
  theta = np.concatenate((np.array([0.05]),np.random.uniform(0.001,0.051,100)))
  kappa = np.concatenate((np.array([0.5]),np.random.uniform(0.5,1.5,100)))
  sigma = np.concatenate((np.array([0.2]),np.minimum(np.sqrt(2*kappa[1:]*theta[1:]),np.random.uniform(0,0.2,100))))
  delta = np.concatenate((np.array([0.0]),np.random.uniform(0,0.03,100)))
  omega = np.random.uniform(0,0.5,100)
  lambda_benchmark = np.array([
        lambda_max_generator(eps, theta[i], theta[i], sigma[i], kappa[i]) for i in range(len(theta))
    ])
  sys.stdout.write(f"\rTrial No.: {i+1}")
  sys.stdout.flush()
  events, marks, prob = simulate_modified_cir_jump(kappa, theta, sigma, T, delta, omega, eps, lambda_benchmark)
  survival_probability += prob
  Payoff_T[i] = np.maximum(np.sum(marks) - 10,0)
print('\n')
bias = np.mean(Payoff_T) - TrueValue
std = np.std(Payoff_T)/np.sqrt(num_trials)
print('\nBias:', bias)
print('Standard deviation:', std)
print('Payoff:', np.mean(Payoff_T))
print('RMSE:',np.sqrt(bias**2 + std**2))
print('Survival Probability of 1st name:',survival_probability/num_trials)

Trial No.: 121.168562306844294
21.44668456579103
22.01913394337859
21.966151138741107
21.935181066238705
21.94617995964893
21.90730665618168
22.76163005491128
23.767509345322956
23.83996679864791
23.82881533265862
23.81017632689118
23.768946352552575
24.39155614758559
23.947397506958886
23.91226316600335
23.721566293527705
23.094490418946464
23.03950465482456
23.516141530371193
23.517182548537765
23.464054160051617
23.68352619470771
23.41381770662364
23.604767553596744
23.093255162669166
23.09236172310908
22.964648061902814
22.965117537334095
23.277418566151685
23.266960385878807
23.242510608233147
23.45889122610621
23.44730999886553
23.409950585486218
23.10725306590607
23.44084632699183
23.896051244196894
23.049173450103332
22.42183669580208
21.64400092840271
21.359869756417776
21.33929466839615
21.31605278949393
21.23905510725815
21.175303229868792
21.262292559703646
21.529474795877256
21.267924909394726
20.98201948619646
20.793212510397478
20.530246910499436
20.366296608032243
20.36

KeyboardInterrupt: 

In [ ]:
# num_trials = 250
# num_simulations = 100
# T = 5
# TrueValue = 5.4494
# Payoff_T = np.zeros(num_trials)
# Payoff = np.zeros(num_simulations)
# eps =  1/num_trials

# mean_survival_probability = np.zeros(num_simulations)
# for j in range(num_simulations):
#   survival_probability = 0
#   theta = np.concatenate((np.array([0.05]),np.random.uniform(0.001,0.051,100)))
#   kappa = np.concatenate((np.array([0.5]),np.random.uniform(0.5,1.5,100)))
#   sigma = np.concatenate((np.array([0.2]),np.minimum(np.sqrt(2*kappa[1:]*theta[1:]),np.random.uniform(0,0.2,100))))
#   delta = np.concatenate((np.array([0.0]),np.random.uniform(0,0.03,100)))
#   omega = np.random.uniform(0,0.5,100)
#   lambda_benchmark = np.array([
#          lambda_max_generator(eps, theta[i], theta[i], sigma[i], kappa[i]) for i in range(len(theta))
#      ])
#   for i in range(num_trials):
#     sys.stdout.write(f"\rSimulation No.: {j+1}| Trial No.: {i+1}")
#     sys.stdout.flush()
#     events, marks, prob = simulate_modified_cir_jump(kappa, theta, sigma, T, delta, omega, eps, lambda_benchmark)
#     survival_probability += prob
#     Payoff_T[i] = np.maximum(np.sum(marks) - 10,0)
#   Payoff[j] = np.mean(Payoff_T)
#   mean_survival_probability[j] = survival_probability/num_trials
# print('\n')
# bias = np.mean(Payoff) - TrueValue
# std = np.std(Payoff)#np.std(Payoff_T)/np.sqrt(num_trials)
# print('\nBias:', bias)
# print('Standard deviation:', std)
# print('Payoff:', np.mean(Payoff))
# print('RMSE:',np.sqrt(bias**2 + std**2))
# print('Survival Probability of 1st name:',mean_survival_probability/num_simulations)

Simulation No.: 1| Trial No.: 46

KeyboardInterrupt: 

In [ ]:
np.mean(mean_survival_probability)

np.float64(34.6018)

In [ ]:
# def simulate_modified_cir_jump(kappa, theta, sigma, T, delta, omega, eps, low=0.0, high=1):

#     events = []
#     marks = []
#     t = 0.0
#     lambda_t = theta
#     gamma  = theta
#     time_grid = [t]

#     lambda_benchmark = np.zeros(len(lambda_t))
#     Dn = np.array([i+1 for i in range(len(omega))])

#     for i in range(len(lambda_t)):
#       lambda_benchmark[i] = lambda_max_generator(eps, lambda_t[i],theta[i], sigma[i], kappa[i])
#     count = 0

#     while t < T:
#       if len(Dn) == 0:
#         break
#       lambda_max = np.zeros(len(lambda_t))

#       indices = np.concatenate(([0], Dn))

#       # Boolean mask where condition is true
#       mask = lambda_t[indices] <= gamma[indices]

#       # Apply the condition
#       lambda_max[indices] = np.where(mask,lambda_benchmark[indices],lambda_t[indices] + lambda_benchmark[indices] - gamma[indices])
#       # for i in np.concatenate((np.array([0]),Dn)):
#       #   if lambda_t[i] <= gamma[i]:
#       #     lambda_max[i] = lambda_benchmark[i]
#       #   else:
#       #     lambda_max[i] = lambda_t[i] + lambda_benchmark[i] - gamma[i]

#       Hepsilon = np.sum(lambda_max[0]*omega[Dn-1]) + np.sum(lambda_max[Dn])

#       tau = np.random.exponential(1 / Hepsilon)
#       t_proposed = t + tau
#       if t_proposed >= T:
#           break

#       lambda_proposedf = np.zeros(len(lambda_t))

#       for i in indices:
#         c = sigma[i]**2 * (1 - np.exp(-kappa[i] * tau)) / (4 * kappa[i])
#         df = 4 * kappa[i] * theta[i] / sigma[i]**2
#         nc = 4 * kappa[i] * lambda_t[i] * np.exp(-kappa[i] * tau) / (sigma[i]**2 * (1 - np.exp(-kappa[i] * tau)))
#         lambda_proposedf[i] = c * ncx2.rvs(df, nc)

#       lambda_proposed = np.sum(lambda_proposedf[0]*omega[Dn-1]) + np.sum(lambda_proposedf[Dn])

#       u = np.random.uniform()
#       if u < np.minimum(lambda_proposed / Hepsilon,1):
#           # Accept: register event and mark
#           t = t_proposed
#           events.append(t)
#           probabilities = (omega[Dn-1]*lambda_proposedf[0] + lambda_proposedf[Dn])/lambda_proposed
#           selected_defaulter = np.random.choice(Dn, p=probabilities)
#           #print(selected_defaulter)
#           Dn =  np.setdiff1d(Dn,selected_defaulter)
#           #print(Dn)
#           mark = np.random.uniform(low, high)
#           marks.append(mark)

#           # Intensity jumps up
#           lambda_t = (lambda_proposedf + delta * mark)
#       else:
#           # Reject: update time
#           t = t_proposed
#           lambda_t = lambda_proposedf

#     return np.array(events), np.array(marks)